In [2]:
import pandas as pd
import numpy as np
import string
import nltk
import itertools
from wordcloud import WordCloud 
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from sklearn.preprocessing import FunctionTransformer

nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
pd.set_option('future.no_silent_downcasting', True)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/jtadych/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jtadych/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jtadych/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
spam_dataset = pd.read_csv('spam.csv', encoding="ISO-8859-1", usecols=[0, 1], names=['Spam', 'Text'], skiprows=1)
spam_dataset['Spam'] = spam_dataset['Spam'].replace(['ham', 'spam'], [0, 1])


In [4]:
def remove_punctation(text):
    return ''.join([word for word in text if word not in string.punctuation])
spam_dataset['Cleaned_Text'] = spam_dataset['Text'].apply(lambda x: remove_punctation(x))

In [5]:
def tokenize(text):

    # Usunięcie wielkich liter
    clean_text = text.lower()

    # Tokenizacja
    tokenized_text = nltk.word_tokenize(clean_text)
    return tokenized_text
spam_dataset['Tokenized_Text'] = spam_dataset['Cleaned_Text'].apply(lambda x: tokenize(x))

In [6]:
stopwords = nltk.corpus.stopwords.words("english")
def remove_stopwords(text):
    return [word for word in text if word not in stopwords]
spam_dataset['WithoutStop_Text'] = spam_dataset['Tokenized_Text'].apply(lambda x: remove_stopwords(x))


In [7]:
stemmer = nltk.PorterStemmer()
def stemming(text):
    stemmed_words = [stemmer.stem(word) for word in text]
    return stemmed_words
spam_dataset['Stemmed_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: stemming(x))


In [8]:
lemmater = nltk.WordNetLemmatizer()
def lemmatizing(text):
    lemmatized_words = [lemmater.lemmatize(word) for word in text]
    return lemmatized_words
spam_dataset['Lemmatized_Text'] = spam_dataset['WithoutStop_Text'].apply(lambda x: lemmatizing(x))

In [9]:
X = spam_dataset['Lemmatized_Text']
y = spam_dataset['Spam']
# Convert continuous labels to discrete classes
y = [int(label) for label in y]

In [12]:
# Podział na trening/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42, stratify=y)


In [13]:
# Funkcja: lista tokenów -> tekst
text_joiner = FunctionTransformer(lambda x: [' '.join(tokens) for tokens in x], validate=False)


In [14]:
# Pipeline bazowy
pipeline = Pipeline([
    ('join', text_joiner),
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(random_state=42))
])

# Trenuj pierwszy model
pipeline.fit(X_train, y_train)

,steps,"[('join', ...), ('tfidf', ...), ...]"
,transform_input,None
,memory,None
,verbose,False
,func,<function <la...t 0x304bd7600>
,inverse_func,None
,validate,False
,accept_sparse,False
,check_inverse,True
,feature_names_out,None
,kw_args,None


In [15]:
# Wyciągnięcie cech i ich ważności
vectorizer = pipeline.named_steps['tfidf']
rf = pipeline.named_steps['rf']
features = vectorizer.get_feature_names_out()
importances = rf.feature_importances_

importance_df = pd.DataFrame({
    'feature': features,
    'importance': importances
}).sort_values(by='importance', ascending=False)

In [16]:
# Selekcja cech
selected_features = importance_df[importance_df['importance'] > 0.001]['feature'].tolist()
print("Wybrane cechy:", selected_features)

Wybrane cechy: ['txt', 'free', 'claim', 'call', 'mobile', 'service', 'reply', 'text', 'prize', 'stop', 'urgent', 'per', 'win', 'nokia', '18', 'tone', '500', 'contact', 'guaranteed', 'customer', '150', 'tc', 'new', '100', 'voucher', '16', 'ringtone', '1000', 'cash', 'draw', 'landline', 'video', 'rate', 'message', 'uk', 'awarded', 'mob', '150p', 'chat', 'orange', 'ur', 'code', 'camera', '5000', 'apply', 'cost', 'dating', 'collection', '250', '86688', 'receive', 'latest', 'quiz', 'offer', 'bonus', 'private', 'send', 'unsubscribe', 'tscs', '150ppm', 'user', 'holiday', '800', '2000', 'please', 'poly', 'network', 'attempt', 'ltd', 'line', 'selected', 'box', 'mobileupd8', 'camcorder', 'award', '150pmsg', '10p', 'identifier', 'pobox', 'sm', 'delivery', 'await', 'po', 'information', 'link', 'get', '150pm', 'sae', 'expires', 'unredeemed', 'collect', 'chance', 'entry', 'charged', 'txts', 'later', 'gift', 'opt', '750', 'immediately', 'eg', '2nd', '0800', 'wap', '350', 'credit', 'im', 'fancy', '200

In [17]:
# Nowy pipeline z ograniczonym słownikiem TF-IDF
pipeline_selected = Pipeline([
    ('join', text_joiner),
    ('tfidf', TfidfVectorizer(vocabulary=selected_features)),
    ('rf', RandomForestClassifier(random_state=42))
])

In [18]:
# GridSearchCV
param_grid = {
    'rf__n_estimators': [50, 100],
    'rf__max_depth': [None, 5],
    'rf__min_samples_split': [2, 4],
}

grid = GridSearchCV(pipeline_selected, param_grid, cv=2, scoring='accuracy')
grid.fit(X_train, y_train)

,estimator,Pipeline(step...m_state=42))])
,param_grid,"{'rf__max_depth': [None, 5], 'rf__min_samples_split': [2, 4], 'rf__n_estimators': [50, 100]}"
,scoring,'accuracy'
,n_jobs,None
,refit,True
,cv,2
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,func,<function <la...t 0x304bd7600>


In [19]:
# Ocena
print("Najlepsze parametry:", grid.best_params_)
y_pred = grid.predict(X_test)
print("Raport klasyfikacji:\n", classification_report(y_test, y_pred))

Najlepsze parametry: {'rf__max_depth': None, 'rf__min_samples_split': 4, 'rf__n_estimators': 50}
Raport klasyfikacji:
               precision    recall  f1-score   support

           0       0.98      0.99      0.99      1592
           1       0.96      0.89      0.92       247

    accuracy                           0.98      1839
   macro avg       0.97      0.94      0.96      1839
weighted avg       0.98      0.98      0.98      1839

